**Text prediction of Shakespeare's literature**

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [6]:
#Mounting drive to access Google Drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/Colab Notebooks/shakespeare.txt','r',encoding='utf-8') as f:
  text = f.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
type(text)

str

In [8]:
text[:1000]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee.\n\n\n                     2\n  When forty winters shall besiege thy brow,\n  And dig deep trenches in thy beauty's field,\n  Thy youth's proud livery so gazed on now,\n  Will be a tattered weed of small worth held:  \n  Then being asked, where all thy beauty lies,\n  Where all the treasure of thy lusty days;\n  To sa

In [9]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [10]:
len(text)

5445609

In [11]:
all_characters = set(text)

In [12]:
all_characters

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

In [13]:
len(all_characters)

84

In [14]:
decoder = dict(enumerate(all_characters))

In [15]:
for pair in enumerate(all_characters): print(pair)

(0, '[')
(1, 'h')
(2, 'f')
(3, ':')
(4, 'W')
(5, '-')
(6, '`')
(7, 'w')
(8, ',')
(9, '?')
(10, 'q')
(11, '7')
(12, '>')
(13, '4')
(14, "'")
(15, 'j')
(16, 'M')
(17, '3')
(18, '0')
(19, 'z')
(20, 'E')
(21, 'O')
(22, 'G')
(23, 'Q')
(24, '&')
(25, 'R')
(26, 'x')
(27, 'l')
(28, '_')
(29, ' ')
(30, 'b')
(31, '6')
(32, '}')
(33, 'I')
(34, 'C')
(35, '\n')
(36, 'd')
(37, 's')
(38, 'n')
(39, 'B')
(40, 'c')
(41, ';')
(42, '5')
(43, ')')
(44, '!')
(45, 'S')
(46, 'v')
(47, '|')
(48, 'm')
(49, 'e')
(50, 'X')
(51, '"')
(52, '2')
(53, '8')
(54, 'J')
(55, 'P')
(56, 'K')
(57, 'u')
(58, 'p')
(59, '.')
(60, 'L')
(61, 'Y')
(62, 'Z')
(63, 'a')
(64, 'N')
(65, 'o')
(66, 'F')
(67, '(')
(68, 'H')
(69, 'A')
(70, '<')
(71, 'D')
(72, '9')
(73, ']')
(74, 'k')
(75, 'i')
(76, 't')
(77, 'y')
(78, 'T')
(79, '1')
(80, 'g')
(81, 'r')
(82, 'U')
(83, 'V')


In [16]:
encoder = {char: ind for ind, char in decoder.items()}

In [17]:
encoder

{'\n': 35,
 ' ': 29,
 '!': 44,
 '"': 51,
 '&': 24,
 "'": 14,
 '(': 67,
 ')': 43,
 ',': 8,
 '-': 5,
 '.': 59,
 '0': 18,
 '1': 79,
 '2': 52,
 '3': 17,
 '4': 13,
 '5': 42,
 '6': 31,
 '7': 11,
 '8': 53,
 '9': 72,
 ':': 3,
 ';': 41,
 '<': 70,
 '>': 12,
 '?': 9,
 'A': 69,
 'B': 39,
 'C': 34,
 'D': 71,
 'E': 20,
 'F': 66,
 'G': 22,
 'H': 68,
 'I': 33,
 'J': 54,
 'K': 56,
 'L': 60,
 'M': 16,
 'N': 64,
 'O': 21,
 'P': 55,
 'Q': 23,
 'R': 25,
 'S': 45,
 'T': 78,
 'U': 82,
 'V': 83,
 'W': 4,
 'X': 50,
 'Y': 61,
 'Z': 62,
 '[': 0,
 ']': 73,
 '_': 28,
 '`': 6,
 'a': 63,
 'b': 30,
 'c': 40,
 'd': 36,
 'e': 49,
 'f': 2,
 'g': 80,
 'h': 1,
 'i': 75,
 'j': 15,
 'k': 74,
 'l': 27,
 'm': 48,
 'n': 38,
 'o': 65,
 'p': 58,
 'q': 10,
 'r': 81,
 's': 37,
 't': 76,
 'u': 57,
 'v': 46,
 'w': 7,
 'x': 26,
 'y': 77,
 'z': 19,
 '|': 47,
 '}': 32}

In [18]:
decoder.items()

dict_items([(0, '['), (1, 'h'), (2, 'f'), (3, ':'), (4, 'W'), (5, '-'), (6, '`'), (7, 'w'), (8, ','), (9, '?'), (10, 'q'), (11, '7'), (12, '>'), (13, '4'), (14, "'"), (15, 'j'), (16, 'M'), (17, '3'), (18, '0'), (19, 'z'), (20, 'E'), (21, 'O'), (22, 'G'), (23, 'Q'), (24, '&'), (25, 'R'), (26, 'x'), (27, 'l'), (28, '_'), (29, ' '), (30, 'b'), (31, '6'), (32, '}'), (33, 'I'), (34, 'C'), (35, '\n'), (36, 'd'), (37, 's'), (38, 'n'), (39, 'B'), (40, 'c'), (41, ';'), (42, '5'), (43, ')'), (44, '!'), (45, 'S'), (46, 'v'), (47, '|'), (48, 'm'), (49, 'e'), (50, 'X'), (51, '"'), (52, '2'), (53, '8'), (54, 'J'), (55, 'P'), (56, 'K'), (57, 'u'), (58, 'p'), (59, '.'), (60, 'L'), (61, 'Y'), (62, 'Z'), (63, 'a'), (64, 'N'), (65, 'o'), (66, 'F'), (67, '('), (68, 'H'), (69, 'A'), (70, '<'), (71, 'D'), (72, '9'), (73, ']'), (74, 'k'), (75, 'i'), (76, 't'), (77, 'y'), (78, 'T'), (79, '1'), (80, 'g'), (81, 'r'), (82, 'U'), (83, 'V')])

In [19]:
encoded_text = np.array([encoder[char] for char in text])

In [20]:
encoded_text[:500]

array([35, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 79, 35, 29, 29, 66, 81, 65, 48, 29,  2, 63, 75,
       81, 49, 37, 76, 29, 40, 81, 49, 63, 76, 57, 81, 49, 37, 29,  7, 49,
       29, 36, 49, 37, 75, 81, 49, 29, 75, 38, 40, 81, 49, 63, 37, 49,  8,
       35, 29, 29, 78,  1, 63, 76, 29, 76,  1, 49, 81, 49, 30, 77, 29, 30,
       49, 63, 57, 76, 77, 14, 37, 29, 81, 65, 37, 49, 29, 48, 75, 80,  1,
       76, 29, 38, 49, 46, 49, 81, 29, 36, 75, 49,  8, 35, 29, 29, 39, 57,
       76, 29, 63, 37, 29, 76,  1, 49, 29, 81, 75, 58, 49, 81, 29, 37,  1,
       65, 57, 27, 36, 29, 30, 77, 29, 76, 75, 48, 49, 29, 36, 49, 40, 49,
       63, 37, 49,  8, 35, 29, 29, 68, 75, 37, 29, 76, 49, 38, 36, 49, 81,
       29,  1, 49, 75, 81, 29, 48, 75, 80,  1, 76, 29, 30, 49, 63, 81, 29,
        1, 75, 37, 29, 48, 49, 48, 65, 81, 77,  3, 35, 29, 29, 39, 57, 76,
       29, 76,  1, 65, 57, 29, 40, 65, 38, 76, 81, 63, 40, 76, 49, 36, 29,
       76, 65, 29, 76,  1

In [21]:
decoder[27]

'l'

In [22]:
def one_hot_encoder(encoded_text,num_uni_chars):

  one_hot = np.zeros((encoded_text.size,num_uni_chars))
  one_hot = one_hot.astype(np.float32)
  one_hot[np.arange(one_hot.shape[0]),encoded_text.flatten()] = 1.0
  one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
  return one_hot

In [23]:
example_text = np.arange(10)
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [24]:
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [25]:
def generate_batches(encoded_text,samp_per_batch=10,seq_len=50):
  #X: encoded text of length seq_len
  #Y: encoded text shifted by one

  #How many chars per batch?
  char_per_batch = samp_per_batch * seq_len

  #How many batches in the given encoded text
  num_batches_avail = int(len(encoded_text)/char_per_batch)

  #Cut off the end of teh encoded text
  encoded_text = encoded_text[:num_batches_avail*char_per_batch]

  encoded_text = encoded_text.reshape((samp_per_batch, -1))

  for n in range(0, encoded_text.shape[1], seq_len):
    x = encoded_text[:,n:n+seq_len]
    y = np.zeros_like(x)

    try:

      y[:,:-1]=x[:,1:]
      y[:,-1]=encoded_text[:,n+seq_len]

    except:
      y[:,:-1]= x[:,1:]
      y[:,-1]=encoded_text[:,0]

    yield x,y

In [26]:
class CharModel(nn.Module):
  def __init__(self, all_chars, num_hidden=256, num_layers=4, drop_prob=0.5,use_gpu=False):

    super().__init__()

    self.drop_prob=drop_prob
    self.num_layers=num_layers
    self.num_hidden=num_hidden
    self.use_gpu = use_gpu

    self.all_chars=all_characters
    self.decoder = dict(enumerate(all_chars))
    self.encoder = {char:ind for ind, char in decoder.items()}

    self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
    self.dropout = nn.Dropout(drop_prob)
    self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

  def forward(self,x,hidden):
    lstm_output, hidden = self.lstm(x, hidden)
    drop_output = self.dropout(lstm_output)
    drop_output = drop_output.contiguous().view(-1, self.num_hidden)
    final_out = self.fc_linear(drop_output)
    return final_out, hidden
  
  def hidden_state(self, batch_size):

      if self.use_gpu:

        hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda(),torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda())
      
      else:

        hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden),torch.zeros(self.num_layers, batch_size, self.num_hidden))

      return hidden

In [27]:
model = CharModel(all_chars = all_characters, num_hidden = 512, num_layers = 3,drop_prob=0.5,use_gpu=True)

In [28]:
total_param = []

for p in model.parameters():
    total_param.append(int(p.numel()))

In [29]:
sum(total_param)

5470292

In [30]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

In [31]:
train_percent = 0.1
train_ind = int(len(encoded_text)*train_percent)

In [32]:
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

**Training the Model**

In [33]:
epochs = 60
batch_size = 100

seq_len = 100
tracker = 0

num_char = max(encoded_text)+1

In [34]:
model.train()

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [35]:
if model.use_gpu: model.cuda()

for i in range(epochs):
  hidden = model.hidden_state(batch_size)

  for x,y in generate_batches(train_data, batch_size,seq_len):
    tracker +=1
    x = one_hot_encoder(x, num_char)

    inputs = torch.from_numpy(x)
    targets = torch.from_numpy(y)

    if model.use_gpu:
      inputs = inputs.cuda()
      targets = targets.cuda()

    hidden = tuple([state.data for state in hidden])

    model.zero_grad()

    lstm_output, hidden = model.forward(inputs, hidden)
    loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())

    loss.backward()

    nn.utils.clip_grad_norm(model.parameters(),max_norm=5)
    optimizer.step()

    if tracker % 25 ==0:

      val_hidden = model.hidden_state(batch_size)
      val_losses = []
  
  model.eval()

  for x,y in generate_batches(val_data, batch_size,seq_len):
        x = one_hot_encoder(x, num_char)
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)

        if model.use_gpu:
             inputs = inputs.cuda()
             targets = targets.cuda()

        val_hidden = tuple([state.data for state in val_hidden])
        lstm_output, val_hidden = model.forward(inputs, val_hidden)
        val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())

        val_losses.append(val_loss.item())

  model.train()

  print(f'Epoch: {i} Step: {tracker} Val Loss:{val_loss.item()}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 0 Step: 54 Val Loss:3.1829493045806885
Epoch: 1 Step: 108 Val Loss:3.1631569862365723
Epoch: 2 Step: 162 Val Loss:2.948288917541504
Epoch: 3 Step: 216 Val Loss:2.7142174243927
Epoch: 4 Step: 270 Val Loss:2.5716471672058105
Epoch: 5 Step: 324 Val Loss:2.3568639755249023
Epoch: 6 Step: 378 Val Loss:2.210324764251709
Epoch: 7 Step: 432 Val Loss:2.132627010345459
Epoch: 8 Step: 486 Val Loss:2.062723159790039
Epoch: 9 Step: 540 Val Loss:2.020033121109009
Epoch: 10 Step: 594 Val Loss:1.9783984422683716
Epoch: 11 Step: 648 Val Loss:1.942792534828186
Epoch: 12 Step: 702 Val Loss:1.91409432888031
Epoch: 13 Step: 756 Val Loss:1.8880290985107422
Epoch: 14 Step: 810 Val Loss:1.8618855476379395
Epoch: 15 Step: 864 Val Loss:1.838640809059143
Epoch: 16 Step: 918 Val Loss:1.82328200340271
Epoch: 17 Step: 972 Val Loss:1.8105007410049438
Epoch: 18 Step: 1026 Val Loss:1.7880192995071411
Epoch: 19 Step: 1080 Val Loss:1.7787673473358154
Epoch: 20 Step: 1134 Val Loss:1.7658336162567139
Epoch: 21 Step

In [36]:
model_name = 'example.net'
torch.save(model.state_dict(),model_name)

In [37]:
model = CharModel(all_chars=all_characters, num_hidden=512, num_layers=3,drop_prob=0.5,use_gpu=True)
model.load_state_dict(torch.load(model_name))

model.eval()

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

**Generating Predictions**

In [54]:
def predict_next_char(model,char, hidden=None, k=1):

  encoded_text=model.encoder[char]
  encoded_text = np.array([[encoded_text]])
  encoded_text=one_hot_encoder(encoded_text, len(model.all_chars))
  inputs = torch.from_numpy(encoded_text)

  if (model.use_gpu):
    inputs = inputs.cuda()
  
  hidden = tuple([state.data for state in hidden])
  lstm_out, hidden = model(inputs, hidden)
  probs = F.softmax(lstm_out, dim=1).data

  if (model.use_gpu):probs = probs.cpu()

  probs, index_positions = probs.topk(k)
  index_positions = index_positions.numpy().squeeze()

  probs = probs.numpy().flatten()
  probs = probs/probs.sum()
  char = np.random.choice(index_positions, p=probs)

  return model.decoder[char], hidden

In [55]:
def generate_text(model, size, seed='The',k=1):

  if(model.use_gpu):
    model.cuda()
  else:
    model.cpu()

  model.eval()
  output_chars = [c for c in seed]
  hidden = model.hidden_state(1)


  for char in seed:
      char, hidden = predict_next_char(model, char, hidden, k=k)
  
  output_chars.append(char)

  for i in range(size):
    char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)
    output_chars.append(char)
  
  return ''.join(output_chars)

In [56]:
print(generate_text(model,1000, seed='The',k=3))

The strong parts
    And things to that which now between his strong,
    And shall the soully works of thee, and to have
    The best of heaven's poor thoughts. The suck and mother,
    Whilst thou that true and strong that should not buy
    The world shall be to hear her. If this thither
    I will speak, being breeds and that he shall
    Take that we speak thee. The men as senten stands
    We will net seek him to him a man from the stattered,
    And by their state out of the wind, and to thy speed,
    That there were strong together. Therefore do not see,
    And that he see himself and man.
  CAESAR. That will not love me all the beauty,
    That should bring him to my brain, and to her,
    And to the sender of his braveres,
    To spoke him by a propertily.
  CLEOPATRA. Is it not so so should hear and means to thee;
    I will not see thee to this, and they say this
    The man's show and treasors. I will be bonged
    The bosom when I send mine own bound of my
    To this w